# 문자 수준 언어 모델-Dinosaurus Island

Dinosaurus Island에 오신 것을 환영합니다! 6 천 5 백만년 전에 공룡이 존재했고 이 과제에서 그들은 돌아 왔습니다. 당신은 특별한 임무를 맡고 있습니다. 선도적인 생물학 연구자들은 새로운 종류의 공룡을 만들어 지구상에 생명을 불어 넣고 있으며, 당신의 임무는 이 공룡들에게 이름을 부여하는 것입니다. 공룡이 이름이 마음에 들지 않으면 광포 할 수 있으니 현명하게 선택하세요!

<table>
<td>
<img src="images/dino.jpg" style="width:250;height:300px;">

</td>

</table>

운 좋게도 딥러닝을 배웠으며 이를 사용하여 하루를 절약 할 수 있습니다. 어시스턴트가 찾을 수 있는 모든 공룡 이름 목록을 수집하여 [dataset](dinos.txt)로 컴파일했습니다. (이전 링크를 클릭하여 자유롭게 살펴보십시오.) 새 공룡 이름을 만들려면 문자 수준 언어 모델을 만들어 새 이름을 생성합니다. 알고리즘은 다른 이름 패턴을 학습하고 무작위로 새 이름을 생성합니다. 이 알고리즘이 여러분과 여러분의 팀을 공룡의 분노로부터 안전하게 지켜 주길 바랍니다!

이 과제를 완료하면 다음을 배울 수 있습니다.

- RNN을 사용하여 처리 할 텍스트 데이터를 저장하는 방법
- 각 시간 단계에서 예측을 샘플링하여 다음 RNN-cell 단위로 전달하여 데이터를 합성하는 방법
- 문자 수준 텍스트 생성 순환 신경망 구축 방법
- 그래디언트 클리핑이 중요한 이유

'rnn_utils'에서 제공 한 일부 함수를 로드하는 것으로 시작합니다. 특히, 이전 할당에서 구현 한 것과 동일한`rnn_forward` 및`rnn_backward`와 같은 함수에 액세스 할 수 있습니다.

In [ ]:
import numpy as np
from utils import *
import random
import pprint

## 1. 문제 설명

### 1.1. 데이터 세트 및 전처리

다음 셀을 실행하여 공룡 이름의 데이터 세트를 읽고 고유 문자 (예 : a-z) 목록을 만들고 데이터 세트와 어휘 크기를 계산합니다.

In [ ]:
data = open('dinos.txt', 'r').read()
data= data.lower()
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print('There are %d total characters and %d unique characters in your data.' % (data_size, vocab_size))

* 문자는 a-z (26 자)와 "\n" (또는 줄바꿈 문자)입니다.
* 이번 과제에서 줄바꿈 문자 "\n"은 우리가 강의에서 논의했던 `<EOS>`(또는 "문장의 끝") 토큰과 유사한 역할을 합니다.
     - 여기서 "\n"은 문장의 끝이 아닌 공룡 이름의 끝을 나타냅니다.
* `char_to_ix` : 아래 셀에서 각 문자를 0-26의 인덱스에 매핑하기 위해 파이썬 사전 (즉, 해시 테이블)을 만듭니다.
* `ix_to_char` : 각 인덱스를 해당 문자에 다시 매핑하는 두 번째 파이썬 사전도 생성합니다.
     - 이것은 소프트맥스 레이어의 확률 분포 출력에서 어떤 문자에 해당하는 인덱스를 파악하는 데 도움이 됩니다.

In [ ]:
chars = sorted(chars)
print(chars)

In [ ]:
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(ix_to_char)

### 1.2. 모델 개요

모델의 구조는 다음과 같습니다.

- 매개 변수 초기화
- 최적화 루프 실행
    - 손실 함수를 계산하기 위한 순방향 전파
    - 손실 함수에 대한 기울기를 계산하기 위한 역 전파
    - 그라디언트가 폭발하지 않도록 그라디언트를 자릅니다.
    - 경사도를 사용하여 경사 하강 법 업데이트 규칙으로 매개변수를 업데이트합니다.
- 학습된 매개변수 반환
    
<img src="images/rnn.png" style="width:450;height:300px;">
<caption><center> **Figure 1**: 이전 노트북 "Building a Recurrent Neural Network-Step by Step"에서 구축 한 것과 유사한 Recurrent Neural Network. </center></caption>

* 각 시간 단계에서 RNN은 이전 문자가 주어지면 다음 문자가 무엇인지 예측하려고 합니다.
* 데이터 세트 $\mathbf{X} = (x^{\langle 1 \rangle}, x^{\langle 2 \rangle}, ..., x^{\langle T_x \rangle})$ 은 훈련 세트의 문자.
* $\mathbf{Y} = (y^{\langle 1 \rangle}, y^{\langle 2 \rangle}, ..., y^{\langle T_x \rangle})$ 은 동일한 문자 목록입니다. 하지만 한 문자 앞으로 이동했습니다.
* 모든 시간 단계에서 $ t $, $y^{\langle t \rangle} = x^{\langle t+1 \rangle}$. 시간 $ t $에서의 예측은 시간 $ t + 1 $에서의 입력과 동일합니다.

## 2. 모델의 구성 요소

이 부분에서는 전체 모델의 두 가지 중요한 블록을 작성합니다.
- 그라디언트 클리핑 : 그라디언트 폭발 방지
- 샘플링 : 캐릭터 생성에 사용되는 기술

그런 다음이 두 기능을 적용하여 모델을 빌드합니다.

### 2.1. 최적화 루프에서 그라디언트 클리핑

이 섹션에서는 최적화 루프 내부에서 호출 할 `clip` 함수를 구현합니다.

####  그래디언트 폭발(Exploding gradients)
* 그래디언트가 매우 크면 "그라디언트 폭발"이라고 합니다.
* 폭발 기울기는 업데이트가 너무 커서 역전파 동안 최적의 값을 "오버슈트"할 수 있기 때문에 훈련 과정를 더 어렵게 만듭니다.

전체 루프 구조는 일반적으로 다음으로 구성됩니다.
* 전방향 패스,
* 비용 계산,
* 역방향 패스,
* 매개 변수 업데이트.

매개 변수를 업데이트하기 전에 그라디언트 클리핑을 수행하여 그라디언트가 "폭발"하지 않는지 확인합니다.

#### 그라디언트 클리핑

아래 연습에서는 그라디언트 딕셔너리를 가져와 필요한 경우 클리핑 된 그라디언트 버전을 반환하는 함수 `clip`을 구현합니다.

* 그라디언트를 자르는 방법에는 여러 가지가 있습니다.
* 간단한 요소 별 클리핑 절차를 사용합니다. 여기서 그라디언트 벡터의 모든 요소는 [-N, N] 범위 사이에 놓이도록 클리핑 됩니다.
* 예를 들어 N = 10 인 경우
     - 범위는 [-10, 10]입니다.
     - 그래디언트 벡터의 구성 요소가 10보다 크면 10으로 설정됩니다.
     - 그래디언트 벡터의 구성 요소가 -10보다 작으면 -10으로 설정됩니다.
     - 구성 요소가 -10에서 10 사이이면 원래 값을 유지합니다.
     
<img src="images/clip.png" style="width:400;height:150px;">
<caption><center>  Figure 2 : 네트워크가 "그라디언트 폭발" 문제에 직면한 경우 그라디언트 클리핑이 있거나 없는 그라디언트 하강의 시각화. </center></caption>

**(1) 연습 과제**:
딕셔너리 'gradients'의 잘린 그래디언트를 반환하려면 아래 함수를 구현하세요.
* 귀하의 함수는 최대 임계값을 취하고 그라디언트의 잘린 버전을 반환합니다.
* [numpy.clip](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.clip.html)을 확인할 수 있습니다.
     - "`out = ...`" 인수를 사용해야 합니다.
     - "`out` "매개 변수를 사용하면 "in-place" 변수를 업데이트 할 수 있습니다.
     - "`out` "인수를 사용하지 않으면 잘린 변수는 "gradient" 변수에 저장되지만 그래디언트 변수`dWax`,`dWaa`,`dWya`,`db`,`dby` 는 업데이트 하지 않습니다. `.

In [ ]:
### GRADED FUNCTION: clip

def clip(gradients, maxValue):
    '''
    Clips the gradients' values between minimum and maximum.
    
    Arguments:
    gradients -- a dictionary containing the gradients "dWaa", "dWax", "dWya", "db", "dby"
    maxValue -- everything above this number is set to this number, and everything less than -maxValue is set to -maxValue
    
    Returns: 
    gradients -- a dictionary with the clipped gradients.
    '''
    
    dWaa, dWax, dWya, db, dby = gradients['dWaa'], gradients['dWax'], gradients['dWya'], gradients['db'], gradients['dby']
   
    ### START CODE HERE ###
    # clip to mitigate exploding gradients, loop over [dWax, dWaa, dWya, db, dby]. (≈2 lines)
    for gradient in [dWaa, dWax, dWya, db, dby]:
        np.clip(gradient, -maxValue, maxValue, out = gradient)
    ### END CODE HERE ###
    
    gradients = {"dWaa": dWaa, "dWax": dWax, "dWya": dWya, "db": db, "dby": dby}
    
    return gradients

In [ ]:
# Test with a maxvalue of 10
maxValue = 10
np.random.seed(3)
dWax = np.random.randn(5,3)*10
dWaa = np.random.randn(5,5)*10
dWya = np.random.randn(2,5)*10
db = np.random.randn(5,1)*10
dby = np.random.randn(2,1)*10
gradients = {"dWax": dWax, "dWaa": dWaa, "dWya": dWya, "db": db, "dby": dby}
gradients = clip(gradients, maxValue)
print("gradients[\"dWaa\"][1][2] =", gradients["dWaa"][1][2])
print("gradients[\"dWax\"][3][1] =", gradients["dWax"][3][1])
print("gradients[\"dWya\"][1][2] =", gradients["dWya"][1][2])
print("gradients[\"db\"][4] =", gradients["db"][4])
print("gradients[\"dby\"][1] =", gradients["dby"][1])

**Expected output:**

```Python
gradients["dWaa"][1][2] = 10.0
gradients["dWax"][3][1] = -10.0
gradients["dWya"][1][2] = 0.29713815361
gradients["db"][4] = [ 10.]
gradients["dby"][1] = [ 8.45833407]
```

In [ ]:
# Test with a maxValue of 5
maxValue = 5
np.random.seed(3)
dWax = np.random.randn(5,3)*10
dWaa = np.random.randn(5,5)*10
dWya = np.random.randn(2,5)*10
db = np.random.randn(5,1)*10
dby = np.random.randn(2,1)*10
gradients = {"dWax": dWax, "dWaa": dWaa, "dWya": dWya, "db": db, "dby": dby}
gradients = clip(gradients, maxValue)
print("gradients[\"dWaa\"][1][2] =", gradients["dWaa"][1][2])
print("gradients[\"dWax\"][3][1] =", gradients["dWax"][3][1])
print("gradients[\"dWya\"][1][2] =", gradients["dWya"][1][2])
print("gradients[\"db\"][4] =", gradients["db"][4])
print("gradients[\"dby\"][1] =", gradients["dby"][1])

**Expected Output: **
```Python
gradients["dWaa"][1][2] = 5.0
gradients["dWax"][3][1] = -5.0
gradients["dWya"][1][2] = 0.29713815361
gradients["db"][4] = [ 5.]
gradients["dby"][1] = [ 5.]
```

### 2.2. 샘플링

이제 모델이 학습되었다고 가정합니다. 새 텍스트 (문자)를 생성하려고 합니다. 생성 과정은 아래 그림에 설명되어 있습니다.

<img src="images/dinos3.png" style="width:500;height:300px;">
<caption><center> Figure 3: 이 그림에서는 모델이 이미 학습되었다고 가정합니다. 첫 번째 단계에서 $x^{\langle 1\rangle} = \vec{0}$ 를 전달하고 네트워크가 한 번에 한 문자 씩 샘플링하도록 합니다. </center></caption>

**(2) 연습 과제** : 아래의 'sample' 함수를 구현하여 캐릭터를 샘플링하세요. 다음 4 단계를 수행해야합니다.

- **1 단계** : 0으로 구성된 "dummy" 벡터를 입력합니다. $x^{\langle 1 \rangle} = \vec{0}$. 
     - 이것은 문자를 생성하기 전의 기본 입력입니다.
     $a^{\langle 0 \rangle} = \vec{0}$ 도 설정했습니다.

- **2 단계** : $a^{\langle 1 \rangle}$ 및 $\hat{y}^{\langle 1 \rangle}$ 를 얻기 위해 한 단계의 순방향 전파를 실행합니다. 다음은 방정식입니다.

은닉 상태 :
$$ a^{\langle t+1 \rangle} = \tanh(W_{ax}  x^{\langle t+1 \rangle } + W_{aa} a^{\langle t \rangle } + b)\tag{1}$$

활성화:
$$ z^{\langle t + 1 \rangle } = W_{ya}  a^{\langle t + 1 \rangle } + b_y \tag{2}$$

예측:
$$ \hat{y}^{\langle t+1 \rangle } = softmax(z^{\langle t + 1 \rangle })\tag{3}$$

- $\hat{y}^{\langle t+1 \rangle }$에 대한 세부 정보 :
    - $\hat{y}^{\langle t+1 \rangle }$ 은 (softmax) 확률 벡터입니다 (항목은 0과 1 사이이고 합은 1입니다).
    - $\hat{y}^{\langle t+1 \rangle}_i$는 "i"로 인덱싱 된 문자가 다음 문자 일 확률을 나타냅니다.
    - 사용할 수 있는`softmax()`함수를 제공합니다.

#### 추가 힌트

- $x^{\langle 1 \rangle}$ 은 코드에서 `x`입니다. 원-핫 벡터를 만들 때 행 수는 고유 문자 수와 같고 열 수는 1과 같은 0으로 구성된 numpy 배열을 만듭니다. 1D 배열이 아니라 2D입니다.
- $a^{\langle 0 \rangle}$ 은 코드에서 `a_prev`입니다. 행 수는 $n_{a}$ 이고 열 수는 1 인 0으로 구성된 numpy 배열입니다.이 배열도 2D 배열입니다. $n_{a}$ 는 $W_{aa}$ 의 열 수를 가져 와서 검색됩니다 (행렬 곱셈 $W_{aa}a^{\langle t \rangle}$ 이 작동하려면 숫자가 일치해야합니다. .
- [numpy.dot](https://docs.scipy.org/doc/numpy/reference/generated/numpy.dot.html)
- [numpy.tanh](https://docs.scipy.org/doc/numpy/reference/generated/numpy.tanh.html)

#### 1D 배열 대신 2D 배열 사용
* $x^{\langle 1 \rangle}$ 및 $a^{\langle 0 \rangle}$ 이 1D 벡터가 아니라 2D 배열임을 강조하는 이유가 궁금 할 것입니다.
* numpy 행렬 곱셈의 경우 2D 행렬에 1D 벡터를 곱하면 1D 배열이됩니다.
* 이것은 동일한 모양을 가질 것으로 예상되는 두 개의 배열을 추가 할 때 문제가 됩니다.
* 차원 수가 다른 두 배열이 함께 추가되면 Python은 서로를 "방송"합니다.
* 다음은 1D 배열과 2D 배열 사용의 차이점을 보여주는 몇 가지 샘플 코드입니다.

In [ ]:
import numpy as np

In [ ]:
matrix1 = np.array([[1,1],[2,2],[3,3]]) # (3,2)
matrix2 = np.array([[0],[0],[0]]) # (3,1) 
vector1D = np.array([1,1]) # (2,) 
vector2D = np.array([[1],[1]]) # (2,1)
print("matrix1 \n", matrix1,"\n")
print("matrix2 \n", matrix2,"\n")
print("vector1D \n", vector1D,"\n")
print("vector2D \n", vector2D)

In [ ]:
print("Multiply 2D and 1D arrays: result is a 1D array\n", 
      np.dot(matrix1,vector1D))
print("Multiply 2D and 2D arrays: result is a 2D array\n", 
      np.dot(matrix1,vector2D))

In [ ]:
print("Adding (3 x 1) vector to a (3 x 1) vector is a (3 x 1) vector\n",
      "This is what we want here!\n", 
      np.dot(matrix1,vector2D) + matrix2)

In [ ]:
print("Adding a (3,) vector to a (3 x 1) vector\n",
      "broadcasts the 1D array across the second dimension\n",
      "Not what we want here!\n",
      np.dot(matrix1,vector1D) + matrix2
     )

- **3 단계** : 샘플링 :
    -이제 $y^{\langle t+1 \rangle}$ 가 있으므로 공룡 이름에서 다음 문자를 선택하려고 합니다. 가장 가능성이 높은 것을 선택하면 모델은 항상 시작 문자가 주어지면 동일한 결과를 생성합니다.
        - 결과를 더 흥미롭게 만들기 위해 np.random.choice를 사용하여 가능성이 높지만 항상 같지는 않은 다음 문자를 선택합니다.
    - 샘플링은 값 그룹에서 값을 선택하는 것으로 각 값은 선택 될 확률이 있습니다.
    - 샘플링을 통해 임의의 값 시퀀스를 생성 할 수 있습니다.
    - $\hat{y}^{\langle t+1 \rangle }$에 지정된 확률 분포에 따라 다음 문자의 인덱스를 선택합니다.
    - 이는 $\hat{y}^{\langle t+1 \rangle }_i = 0.16$ 이면 16 % 확률로 인덱스 "i"를 선택한다는 의미입니다.
    - [np.random.choice](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.random.choice.html)를 사용할 수 있습니다.

    `np.random.choice()`사용 방법의 예 :
    ```python
    np.random.seed(0)
    probs = np.array([0.1, 0.0, 0.7, 0.2])
    idx = np.random.choice([0, 1, 2, 3] p = probs)
    ```
    - 이는 분포에 따라 색인 (`idx`)을 선택한다는 의미입니다.

    $P(index = 0) = 0.1, P(index = 1) = 0.0, P(index = 2) = 0.7, P(index = 3) = 0.2$.

    - 'p'로 설정된 값은 1D 벡터로 설정되어야합니다.
    - 또한 코드에서`y` 인 $\hat{y}^{\langle t+1 \rangle}$ 는 2D 배열입니다.

##### 추가 힌트
- [range](https://docs.python.org/3/library/functions.html#func-range)
- [numpy.ravel](https://docs.scipy.org/doc/numpy/reference/generated/numpy.ravel.html) 다차원 배열을 취하고 그 내용을 1D 벡터 내부에 반환합니다.
```Python
arr = np.array([[1,2],[3,4]])
print("arr")
print(arr)
print("arr.ravel()")
print(arr.ravel())
```
Output:
```Python
arr
[[1 2]
 [3 4]]
arr.ravel()
[1 2 3 4]
```

- `append` 는 "in-place" 작업입니다. 즉, 다음과 같이 하지 마십시오.:
```Python
fun_hobbies = fun_hobbies.append('learning')  ## Doesn't give you what you want
```

- **4 단계** : $x^{\langle t \rangle }$로 업데이트 합니다.
     - `sample()`에서 구현하는 마지막 단계는 현재 $x^{\langle t \rangle }$을 저장하고있는 변수 `x` 를 $x^{\langle t + 1 \rangle }$ 값으로 업데이트하는 것입니다. 
     - 예측으로 선택한 캐릭터에 해당하는 원-핫 벡터를 생성하여 $x^{\langle t + 1 \rangle }$ 을 나타냅니다.
     - 그런 다음 1 단계에서 $x^{\langle t + 1 \rangle }$을 전달하고 공룡 이름의 끝에 도달했음을 나타내는 "\n"문자를 얻을 때까지 프로세스를 계속 반복합니다.

##### 추가 힌트
- 새로운 원-핫 벡터로 설정하기 전에`x` 를 재설정하려면 모든 값을 0으로 설정해야 합니다.
     - 새 numpy 배열을 만들 수 있습니다.: [numpy.zeros](https://docs.scipy.org/doc/numpy/reference/generated/numpy.zeros.html)
    - 또는 모든 값을 단일 숫자로 채웁니다.: [numpy.ndarray.fill](https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.fill.html)

In [ ]:
# GRADED FUNCTION: sample

def sample(parameters, char_to_ix, seed):
    """
    Sample a sequence of characters according to a sequence of probability distributions output of the RNN

    Arguments:
    parameters -- python dictionary containing the parameters Waa, Wax, Wya, by, and b. 
    char_to_ix -- python dictionary mapping each character to an index.
    seed -- used for grading purposes. Do not worry about it.

    Returns:
    indices -- a list of length n containing the indices of the sampled characters.
    """
    
    # Retrieve parameters and relevant shapes from "parameters" dictionary
    Waa, Wax, Wya, by, b = parameters['Waa'], parameters['Wax'], parameters['Wya'], parameters['by'], parameters['b']
    vocab_size = by.shape[0]
    n_a = Waa.shape[1]
    
    ### START CODE HERE ###
    # Step 1: Create the a zero vector x that can be used as the one-hot vector 
    # representing the first character (initializing the sequence generation). (≈1 line)
    x = np.zeros((vocab_size, 1))
    # Step 1': Initialize a_prev as zeros (≈1 line)
    a_prev = np.zeros((n_a, 1))
    
    # Create an empty list of indices, this is the list which will contain the list of indices of the characters to generate (≈1 line)
    indices = []
    
    # idx is the index of the one-hot vector x that is set to 1
    # All other positions in x are zero.
    # We will initialize idx to -1
    idx = -1 
    
    # Loop over time-steps t. At each time-step:
    # sample a character from a probability distribution 
    # and append its index (`idx`) to the list "indices". 
    # We'll stop if we reach 50 characters 
    # (which should be very unlikely with a well trained model).
    # Setting the maximum number of characters helps with debugging and prevents infinite loops. 
    counter = 0
    newline_character = char_to_ix['\n']
    
    while (idx != newline_character and counter != 50):
        
        # Step 2: Forward propagate x using the equations (1), (2) and (3)
        a = np.tanh(np.dot(Wax, x) + np.dot(Waa, a_prev) + b)
        z = np.dot(Wya, a) + by
        y = softmax(z)
        
        # for grading purposes
        np.random.seed(counter+seed) 
        
        # Step 3: Sample the index of a character within the vocabulary from the probability distribution y
        # (see additional hints above)
        idx = np.random.choice(range(len(y)), p = y.ravel())

        # Append the index to "indices"
        indices.append(idx)
        
        # Step 4: Overwrite the input x with one that corresponds to the sampled index `idx`.
        # (see additional hints above)
        x = np.zeros((vocab_size, 1))
        x[idx] = 1
        
        # Update "a_prev" to be "a"
        a_prev = a
        
        # for grading purposes
        seed += 1
        counter +=1
        
    ### END CODE HERE ###

    if (counter == 50):
        indices.append(char_to_ix['\n'])
    
    return indices

In [ ]:
np.random.seed(2)
_, n_a = 20, 100
Wax, Waa, Wya = np.random.randn(n_a, vocab_size), np.random.randn(n_a, n_a), np.random.randn(vocab_size, n_a)
b, by = np.random.randn(n_a, 1), np.random.randn(vocab_size, 1)
parameters = {"Wax": Wax, "Waa": Waa, "Wya": Wya, "b": b, "by": by}


indices = sample(parameters, char_to_ix, 0)
print("Sampling:")
print("list of sampled indices:\n", indices)
print("list of sampled characters:\n", [ix_to_char[i] for i in indices])

**Expected output:**

```Python
Sampling:
list of sampled indices:
 [12, 17, 24, 14, 13, 9, 10, 22, 24, 6, 13, 11, 12, 6, 21, 15, 21, 14, 3, 2, 1, 21, 18, 24, 7, 25, 6, 25, 18, 10, 16, 2, 3, 8, 15, 12, 11, 7, 1, 12, 10, 2, 7, 7, 11, 17, 24, 12, 13, 24, 0]
list of sampled characters:
 ['l', 'q', 'x', 'n', 'm', 'i', 'j', 'v', 'x', 'f', 'm', 'k', 'l', 'f', 'u', 'o', 'u', 'n', 'c', 'b', 'a', 'u', 'r', 'x', 'g', 'y', 'f', 'y', 'r', 'j', 'p', 'b', 'c', 'h', 'o', 'l', 'k', 'g', 'a', 'l', 'j', 'b', 'g', 'g', 'k', 'q', 'x', 'l', 'm', 'x', '\n']
```

* 시간이 지남에 따라 Coursera 플랫폼의 백엔드가 업데이트되면 (numpy 버전이 업데이트 될 수 있음) 샘플링 된 인덱스 및 샘플링 된 문자의 실제 목록이 변경 될 수 있습니다.
* 위에 제공된 지침에 따라 오류없이 출력을 얻는 경우 출력이 예상 출력과 일치하지 않더라도 루틴이 정확할 수 있습니다. 채점자에게 과제를 제출하여 정확성을 확인합니다.

## 3. 언어 모델 구축

텍스트 생성을 위한 문자 수준 언어 모델을 구축 할 때입니다.


### 3.1. 경사하강 법

* 이 섹션에서는 확률적 경사하강 법 (잘린 경사 포함)의 한 단계를 수행하는 함수를 구현합니다.
* 학습 예제를 한 번에 하나씩 진행하므로 최적화 알고리즘은 확률적 경사하강 법이 됩니다.

다음은 RNN에 대한 일반적인 최적화 루프의 단계입니다.

- 손실을 계산하기 위해 RNN을 통해 순방향 전파
- 시간을 통해 역 전파되어 매개 변수에 대한 손실의 기울기를 계산합니다.
- 그라디언트 자르기
- 경사하강 법을 사용하여 매개 변수 업데이트

**(2) 연습 과제** : 최적화 프로세스를 구현합니다 (확률적 경사하강 법의 한 단계).

다음과 같은 함수가 제공됩니다.:

```python
def rnn_forward(X, Y, a_prev, parameters):
    """ Performs the forward propagation through the RNN and computes the cross-entropy loss.
    It returns the loss' value as well as a "cache" storing values to be used in backpropagation."""
    ....
    return loss, cache
    
def rnn_backward(X, Y, parameters, cache):
    """ Performs the backward propagation through time to compute the gradients of the loss with respect
    to the parameters. It returns also all the hidden states."""
    ...
    return gradients, a

def update_parameters(parameters, gradients, learning_rate):
    """ Updates parameters using the Gradient Descent Update Rule."""
    ...
    return parameters
```

Recall that you previously implemented the `clip` function:

```Python
def clip(gradients, maxValue)
    """Clips the gradients' values between minimum and maximum."""
    ...
    return gradients
```

#### 매개 변수

* `parameters` 가 `optimize` 함수의 반환 된 값 중 하나가 아니더라도 `parameters` 사전 내의 가중치 및 편향은 최적화에 의해 업데이트됩니다. `parameters` 딕셔너리는 참조에 의해 함수에 전달되므로 이 딕셔너리을 변경하면 함수 외부에서 액세스하더라도 `parameters` 딕셔너리가 변경됩니다.
* 파이썬 사전과 목록은 "참조에 의한 전달"입니다. 즉, 사전을 함수에 전달하고 함수 내에서 사전을 수정하면 동일한 사전이 변경됩니다 (사전의 사본이 아님).

In [ ]:
# GRADED FUNCTION: optimize

def optimize(X, Y, a_prev, parameters, learning_rate = 0.01):
    """
    Execute one step of the optimization to train the model.
    
    Arguments:
    X -- list of integers, where each integer is a number that maps to a character in the vocabulary.
    Y -- list of integers, exactly the same as X but shifted one index to the left.
    a_prev -- previous hidden state.
    parameters -- python dictionary containing:
                        Wax -- Weight matrix multiplying the input, numpy array of shape (n_a, n_x)
                        Waa -- Weight matrix multiplying the hidden state, numpy array of shape (n_a, n_a)
                        Wya -- Weight matrix relating the hidden-state to the output, numpy array of shape (n_y, n_a)
                        b --  Bias, numpy array of shape (n_a, 1)
                        by -- Bias relating the hidden-state to the output, numpy array of shape (n_y, 1)
    learning_rate -- learning rate for the model.
    
    Returns:
    loss -- value of the loss function (cross-entropy)
    gradients -- python dictionary containing:
                        dWax -- Gradients of input-to-hidden weights, of shape (n_a, n_x)
                        dWaa -- Gradients of hidden-to-hidden weights, of shape (n_a, n_a)
                        dWya -- Gradients of hidden-to-output weights, of shape (n_y, n_a)
                        db -- Gradients of bias vector, of shape (n_a, 1)
                        dby -- Gradients of output bias vector, of shape (n_y, 1)
    a[len(X)-1] -- the last hidden state, of shape (n_a, 1)
    """
    
    ### START CODE HERE ###
    
    # Forward propagate through time (≈1 line)
    loss, cache = rnn_forward(X, Y, a_prev, parameters)
    
    # Backpropagate through time (≈1 line)
    gradients, a = rnn_backward(X, Y, parameters, cache)
    
    # Clip your gradients between -5 (min) and 5 (max) (≈1 line)
    gradients = clip(gradients, maxValue)
    
    # Update parameters (≈1 line)
    parameters = update_parameters(parameters, gradients, learning_rate)
    
    ### END CODE HERE ###
    
    return loss, gradients, a[len(X)-1]

In [ ]:
np.random.seed(1)
vocab_size, n_a = 27, 100
a_prev = np.random.randn(n_a, 1)
Wax, Waa, Wya = np.random.randn(n_a, vocab_size), np.random.randn(n_a, n_a), np.random.randn(vocab_size, n_a)
b, by = np.random.randn(n_a, 1), np.random.randn(vocab_size, 1)
parameters = {"Wax": Wax, "Waa": Waa, "Wya": Wya, "b": b, "by": by}
X = [12,3,5,11,22,3]
Y = [4,14,11,22,25, 26]

loss, gradients, a_last = optimize(X, Y, a_prev, parameters, learning_rate = 0.01)
print("Loss =", loss)
print("gradients[\"dWaa\"][1][2] =", gradients["dWaa"][1][2])
print("np.argmax(gradients[\"dWax\"]) =", np.argmax(gradients["dWax"]))
print("gradients[\"dWya\"][1][2] =", gradients["dWya"][1][2])
print("gradients[\"db\"][4] =", gradients["db"][4])
print("gradients[\"dby\"][1] =", gradients["dby"][1])
print("a_last[4] =", a_last[4])

** Expected output:**

```Python
Loss = 126.503975722
gradients["dWaa"][1][2] = 0.194709315347
np.argmax(gradients["dWax"]) = 93
gradients["dWya"][1][2] = -0.007773876032
gradients["db"][4] = [-0.06809825]
gradients["dby"][1] = [ 0.01538192]
a_last[4] = [-1.]
```

### 3.2. 모델 훈련

* 공룡 이름 데이터 세트가 주어지면 데이터 세트의 각 줄 (이름 하나)을 하나의 훈련 예제로 사용합니다.
* 확률적 경사하강 법의 100 단계마다 무작위로 선택된 10 개의 이름을 샘플링하여 알고리즘이 어떻게 작동하는지 확인합니다.
* 확률적 경사하강 법이 무작위 순서로 예제를 방문하도록 데이터 세트를 섞는 것을 잊지 마십시오.

**(3) 연습 과제** : 지침에 따라 `model()`을 구현하세요. `examples[index]`에 공룡 이름 (문자열)이 하나 포함 된 경우 예제 (X, Y)를 만들려면 다음을 사용할 수 있습니다.

##### 색인  `idx`를 예제 목록에 설정

* for-loop를 사용하여 "예제"목록에서 셔플 된 공룡 이름 목록을 살펴보세요.
* 100 개의 예제가 있고 for 루프가 인덱스를 100으로 증가시키는 경우, j가 100, 101, 등 일 때 모델에 예제를 계속 공급할 수 있도록 인덱스 주기를 0으로 되돌리는 방법을 생각해보십시오.
* 힌트 : 101을 100으로 나눈 값은 0이고 나머지는 1입니다.
* `%`는 파이썬의 모듈러스 연산자입니다.

##### 예제 목록에서 단일 예제 추출
* `single_example` : 이전에 설정한 `idx` 인덱스를 사용하여 예제 목록에서 한 단어를 가져옵니다.

##### 문자열을 문자 목록으로 변환 : `single_example_chars`
* `single_example_chars` : 문자열은 문자 목록입니다.
* 목록 이해 (for 루프보다 권장 됨)를 사용하여 문자 목록을 생성 할 수 있습니다.

```Python
str = 'I love learning'
list_of_chars = [c for c in str]
print(list_of_chars)
```

```
['I', ' ', 'l', 'o', 'v', 'e', ' ', 'l', 'e', 'a', 'r', 'n', 'i', 'n', 'g']
```

##### 문자 목록을 정수 목록으로 변환 : `single_example_ix`
* 각 문자와 관련된 색인 번호를 포함하는 목록을 만듭니다.
* 사전`char_to_ix` 사용
* 문자열에서 문자 목록을 가져 오는 데 사용되는 목록 이해와 결합 할 수 있습니다.
* 이것은 학습자가 함수의 각 단계를 명확히하는 데 도움이 되는 별도의 코드 줄입니다.

##### 입력 문자 목록 만들기 : `X`
* `rnn_forward` 는 `None` 값을 플래그로 사용하여 입력 벡터를 제로 벡터로 설정합니다.
* 입력 문자 목록 앞에 '없음'값을 추가합니다.
* 목록에 값을 추가하는 방법은 여러 가지가 있습니다. 한 가지 방법은 두 개의 목록을 함께 추가하는 것입니다.`['a'] + ['b']`

##### 개행 문자 `ix_newline` 의 정수 표현을 가져옵니다.
* `ix_newline` : 개행 문자는 공룡 이름의 끝을 나타냅니다.
     - 개행 문자` '\n'`의 정수 표현을 얻습니다.
     - `char_to_ix` 사용

##### 레이블 목록 설정 (문자의 정수 표시) : `Y`
* 목표는 이름의 다음 문자를 예측하도록 RNN을 훈련시키는 것이므로 레이블은 입력 'X'의 문자보다 한 번 앞선 문자 목록입니다.
     - 예를 들어`Y[0]`에는`X[1]`과 동일한 값이 포함됩니다.
* RNN은 마지막 문자에서 줄 바꿈을 예측해야하므로 레이블 끝에 ix_newline을 추가합니다.
     - 줄 바꿈 문자의 정수 표현을`Y` 끝에 추가합니다.
     - '추가'는 내부 작업입니다.
     - 두 개의 목록을 함께 추가하는 것이 더 쉬울 수 있습니다.

In [ ]:
# GRADED FUNCTION: model

def model(data, ix_to_char, char_to_ix, num_iterations = 35000, n_a = 50, dino_names = 7, vocab_size = 27):
    """
    Trains the model and generates dinosaur names. 
    
    Arguments:
    data -- text corpus
    ix_to_char -- dictionary that maps the index to a character
    char_to_ix -- dictionary that maps a character to an index
    num_iterations -- number of iterations to train the model for
    n_a -- number of units of the RNN cell
    dino_names -- number of dinosaur names you want to sample at each iteration. 
    vocab_size -- number of unique characters found in the text (size of the vocabulary)
    
    Returns:
    parameters -- learned parameters
    """
    
    # Retrieve n_x and n_y from vocab_size
    n_x, n_y = vocab_size, vocab_size
    
    # Initialize parameters
    parameters = initialize_parameters(n_a, n_x, n_y)
    
    # Initialize loss (this is required because we want to smooth our loss)
    loss = get_initial_loss(vocab_size, dino_names)
    
    # Build list of all dinosaur names (training examples).
    with open("dinos.txt") as f:
        examples = f.readlines()
    examples = [x.lower().strip() for x in examples]
    
    # Shuffle list of all dinosaur names
    np.random.seed(0)
    np.random.shuffle(examples)
    
    # Initialize the hidden state of your LSTM
    a_prev = np.zeros((n_a, 1))
    
    # Optimization loop
    for j in range(num_iterations):
        
        ### START CODE HERE ###
        
        # Set the index `idx` (see instructions above)
        idx = j % len(examples)
        
#         # Set the input X (see instructions above)
#         single_example = examples[idx]
# #         single_example_chars = [c for c in single_example]
#         single_example_ix = [char_to_ix[ch] for ch in examples[idx]] 
#         X = [None] + [single_example_ix]
        
#         # Set the labels Y (see instructions above)
#         ix_newline = char_to_ix['\n']
#         Y = X[1:] + [char_to_ix["\n"]]
        
        X = [None] + [char_to_ix[ch] for ch in examples[idx]] 
        Y = X[1:] + [char_to_ix["\n"]]
        
        # Perform one optimization step: Forward-prop -> Backward-prop -> Clip -> Update parameters
        # Choose a learning rate of 0.01
        curr_loss, gradients, a_prev = optimize(X, Y, a_prev, parameters, learning_rate = 0.01)
        
        ### END CODE HERE ###
        
        # Use a latency trick to keep the loss smooth. It happens here to accelerate the training.
        loss = smooth(loss, curr_loss)

        # Every 2000 Iteration, generate "n" characters thanks to sample() to check if the model is learning properly
        if j % 2000 == 0:
            
            print('Iteration: %d, Loss: %f' % (j, loss) + '\n')
            
            # The number of dinosaur names to print
            seed = 0
            for name in range(dino_names):
                
                # Sample indices and print them
                sampled_indices = sample(parameters, char_to_ix, seed)
                print_sample(sampled_indices, ix_to_char)
                
                seed += 1  # To get the same result (for grading purposes), increment the seed by one. 
      
            print('\n')
        
    return parameters

다음 셀을 실행하면 첫 번째 반복에서 모델이 임의의 문자를 출력하는 것을 관찰해야 합니다. 수천 번의 반복 후에 모델은 합리적으로 보이는 이름을 생성하는 방법을 배워야 합니다.

In [ ]:
parameters = model(data, ix_to_char, char_to_ix)

**Expected Output**

모델의 출력은 다르게 보일 수 있지만 다음과 같습니다.:

```Python
Iteration: 34000, Loss: 22.447230

Onyxipaledisons
Kiabaeropa
Lussiamang
Pacaeptabalsaurus
Xosalong
Eiacoteg
Troia
```

## 결론

알고리즘이 훈련이 끝날 무렵 그럴듯한 공룡 이름을 생성하기 시작했음을 알 수 있습니다. 처음에는 임의의 캐릭터를 생성했지만 마지막에는 멋진 엔딩으로 공룡 이름을 볼 수있었습니다. 알고리즘을 더 오래 실행하고 하이퍼 파라미터를 사용하여 더 나은 결과를 얻을 수 있는지 확인하십시오. 우리의 구현은 `maconucon`, `marloralus` 및 `macingsersaurus`와 같은 정말 멋진 이름을 생성했습니다. 여러분의 모델은 공룡 이름이 'saurus', 'don', 'aura', 'tor' 등으로 끝나는 경향이 있다는 것을 알게되기를 바랍니다.

모델이 멋지지 않은 이름을 생성하는 경우 모델을 완전히 비난하지 마십시오. 실제 공룡 이름이 모두 멋지게 들리는 것은 아닙니다. (예를 들어, `dromaeosauroides`는 실제 공룡 이름이며 훈련 세트에 있습니다.)하지만이 모델은 가장 멋진 것을 선택할 수있는 후보 세트를 제공해야합니다!

이 할당은 비교적 작은 데이터 세트를 사용했기 때문에 CPU에서 RNN을 빠르게 훈련 할 수있었습니다. 영어 모델을 학습하려면 훨씬 더 큰 데이터 세트가 필요하며 일반적으로 훨씬 더 많은 계산이 필요하며 GPU에서 여러 시간 동안 실행할 수 있습니다. 우리는 공룡 이름을 꽤 오랫동안 사용했으며 지금까지 우리가 가장 좋아하는 이름은 위대하고 무패하며 치열한 망고 사우루스입니다!

<img src="images/mangosaurus.jpeg" style="width:250;height:300px;">

## 4. 셰익스피어처럼 글쓰기

이 노트의 나머지 부분은 선택 사항이며 등급이 매겨지지는 않지만 재미 있고 유익한 내용이므로 어쨌든 해주시기를 바랍니다.

비슷하지만 더 복잡한 작업은 셰익스피어시를 만드는 것입니다. 공룡 이름 데이터 세트에서 배우는 대신 셰익스피어시 모음을 사용할 수 있습니다. LSTM 셀을 사용하면 텍스트의 여러 문자에 걸쳐있는 장기 종속성을 학습 할 수 있습니다. 예를 들어 시퀀스 어딘가에 나타나는 문자가 시퀀스에서 훨씬 나중에 다른 문자가되어야하는 것에 영향을 줄 수 있습니다. 공룡 이름은 이름이 매우 짧기 때문에 이러한 장기적인 의존성은 덜 중요했습니다.


<img src="images/shakespeare.jpg" style="width:500;height:400px;">
<caption><center> Let's become poets! </center></caption>

Keras로 셰익스피어시 생성기를 구현했습니다. 다음 셀을 실행하여 필요한 패키지 및 모델을로드하십시오. 이 작업은 몇 분 정도 걸릴 수 있습니다.

In [ ]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Model, load_model, Sequential
from keras.layers import Dense, Activation, Dropout, Input, Masking
from keras.layers import LSTM
from keras.utils.data_utils import get_file
from keras.preprocessing.sequence import pad_sequences
from shakespeare_utils import *
import sys
import io

시간을 절약하기 위해 우리는 이미 [*"The Sonnets"*](shakespeare.txt)라는 셰익스피어시 모음집에서 ~ 1000 epoch에 대한 모델을 훈련했습니다.

한 세대 더 모델을 훈련시켜 봅시다. 한 시대에 대한 학습이 완료되면 몇 분 정도 걸립니다. 입력을 요청하는 `generate_output` 을 실행할 수 있습니다 (`<`40 자). 시는 문장으로 시작하고 RNN-Shakespeare가 나머지시를 완성합니다! 예를 들어, "Forsooth this maketh no sense"(따옴표를 입력하지 마십시오)를 시도하십시오. 끝에 공백을 포함하는지 여부에 따라 결과도 다를 수 있습니다. 두 가지 방법을 모두 시도하고 다른 입력도 시도해보세요.

In [ ]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y, batch_size=128, epochs=1, callbacks=[print_callback])

In [ ]:
# Run this cell to try with different inputs without having to re-train the model 
generate_output()

RNN-Shakespeare 모델은 공룡 이름으로 만든 모델과 매우 유사합니다. 유일한 주요 차이점은 다음과 같습니다.
- 기본 RNN 대신 LSTM을 사용하여 장거리 종속성 캡처
- 모델이 더 깊고 적층 된 LSTM 모델 (2 층)
- Python 대신 Keras를 사용하여 코드 단순화

자세한 내용은 GitHub에서 Keras Team의 텍스트 생성 구현 (https://github.com/keras-team/keras/blob/master/examples/lstm_text_generation.py)을 확인할 수도 있습니다.

이 노트를 완성한 것을 축하합니다!

**참조** :
- 이 연습은 Andrej Karpathy의 구현에서 영감을 얻었습니다 : https://gist.github.com/karpathy/d4dee566867f8291f086. 텍스트 생성에 대해 자세히 알아 보려면 Karpathy의 [blog post](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)도 확인하세요.
-셰익스피어시 생성기의 경우 Keras 팀 (https://github.com/keras-team/keras/blob/master/examples/lstm_text_generation.py)의 LSTM 텍스트 생성기 구현을 기반으로 구현되었습니다.